In [1]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [3]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

# model

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [5]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)

    return x

In [6]:
def build_mobilenetv2_unet(input_shape):    ## (512, 512, 3)
    """ Input """
    inputs = Input(shape=input_shape)

    """ Pre-trained MobileNetV2 """
    encoder = MobileNetV2(include_top=False, weights="imagenet",
        input_tensor=inputs, alpha=1.4)

    """ Encoder """
    s1 = encoder.get_layer("input_1").output                ## (512 x 512)
    s2 = encoder.get_layer("block_1_expand_relu").output    ## (256 x 256)
    s3 = encoder.get_layer("block_3_expand_relu").output    ## (128 x 128)
    s4 = encoder.get_layer("block_6_expand_relu").output    ## (64 x 64)

    """ Bridge """
    b1 = encoder.get_layer("block_13_expand_relu").output   ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 256)                         ## (64 x 64)
    d2 = decoder_block(d1, s3, 128)                         ## (128 x 128)
    d3 = decoder_block(d2, s2, 64)                         ## (256 x 256)
    d4 = decoder_block(d3, s1, 32)                          ## (512 x 512)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="MobileNetV2_U-Net")
    return model

In [7]:
input_shape = (256, 256, 3)
model = build_mobilenetv2_unet(input_shape)
model.summary()

17831080/17831080 [==============================] - 0s 0us/step
Model: "MobileNetV2_U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 128, 128, 48)         1296      ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 128, 128, 48)         192       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU

# train

In [8]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
# from data import load_data, tf_dataset
# from model import build_model

In [9]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


In [10]:
path = "/kaggle/input/kvasirseg/Kvasir-SEG/"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

print(len(train_x),len(train_y),len(valid_x),len(valid_y),len(test_x),len(test_y))



800 800 100 100 100 100


In [11]:
## Hyperparameters
batch = 8
lr = 1e-4
epochs = 400

train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

for x,y in train_dataset:
    print(x.shape,y.shape)
    break;

(8, 256, 256, 3) (8, 256, 256, 1)


In [12]:
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

callbacks = [
        ModelCheckpoint("files/model.h5",monitor="val_acc", mode="max", 
                         save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 15, 
                         min_delta = 0.001,min_lr = 1e-10,verbose = 1),
        CSVLogger("files/data.csv"),
        TensorBoard()
    ]



In [13]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1

model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks)

Epoch 1/400
100/100 [==============================] - ETA: 0s - loss: 0.2846 - acc: 0.8956 - recall: 0.5729 - precision: 0.7325 - iou: 0.3192
Epoch 1: val_acc improved from -inf to 0.63365, saving model to files/model.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 76s 293ms/step - loss: 0.2846 - acc: 0.8956 - recall: 0.5729 - precision: 0.7325 - iou: 0.3192 - val_loss: 1.1364 - val_acc: 0.6336 - val_recall: 0.9707 - val_precision: 0.3203 - val_iou: 0.2497 - lr: 1.0000e-04
Epoch 2/400
100/100 [==============================] - ETA: 0s - loss: 0.1401 - acc: 0.9534 - recall: 0.8308 - precision: 0.9019 - iou: 0.5417
Epoch 2: val_acc improved from 0.63365 to 0.89512, saving model to files/model.h5
100/100 [==============================] - 18s 181ms/step - loss: 0.1401 - acc: 0.9534 - recall: 0.8308 - precision: 0.9019 - iou: 0.5417 - val_loss: 0.3396 - val_acc: 0.8951 - val_recall: 0.8885 - val_precision: 0.6634 - val_iou: 0.3913 - lr: 1.0000e-04
Epoch 3/400
100/100 [==============================] - ETA: 0s - loss: 0.0927 - acc: 0.9700 - recall: 0.8954 - precision: 0.9508 - iou: 0.6478
Epoch 3: val_acc improved from 0.89512 to 0.93628, saving model to files/model.h5
100/100 [==============================]

In [14]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
# from data import load_data, tf_dataset
# from train import iou


In [15]:
def test_read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def test_read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    return x

def test_mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask


In [16]:
test_dataset = tf_dataset(test_x, test_y, batch=batch)
for x,y in test_dataset:
    print(x.shape,y.shape)
    break;

(8, 256, 256, 3) (8, 256, 256, 1)


In [17]:
test_steps = (len(test_x)//batch)
if len(test_x) % batch != 0:
    test_steps += 1

# with CustomObjectScope({'iou': iou}):
#     model = tf.keras.models.load_model("files/model.h5")

model.evaluate(test_dataset, steps=test_steps)

for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
    x = test_read_image(x)
    y = test_read_mask(y)
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
    h, w, _ = x.shape
    white_line = np.ones((h, 10, 3)) * 255.0

    all_images = [
            x * 255.0, white_line,
            test_mask_parse(y), white_line,
            test_mask_parse(y_pred) * 255.0
        ]
    image = np.concatenate(all_images, axis=1)
    cv2.imwrite(f"results/{i}.png", image)

13/13 [==============================] - 2s 192ms/step - loss: 0.2729 - acc: 0.9512 - recall: 0.8230 - precision: 0.9140 - iou: 0.7729


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  1%|          | 1/100 [00:02<03:27,  2.09s/it]

1/1 [==============================] - 0s 23ms/step


...

 97%|█████████▋| 97/100 [00:09<00:00, 11.76it/s]

1/1 [==============================] - 0s 22ms/step


 99%|█████████▉| 99/100 [00:09<00:00, 12.04it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 100/100 [00:09<00:00, 10.03it/s]
